In [ ]:
!pip install -U transformers accelerate bitsandbytes

In [2]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [3]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-10-14 10:13:28--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-10-14 10:13:28 (63.9 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
import requests
import minsearch
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
doce_response = requests.get(docs_url)
documents_raw = doce_response.json()

documents = []

for course in documents_raw:
  course_name = course['course']

  for doc in course ['documents']:
    doc['course'] = course_name
    documents.append(doc)

index = minsearch.Index(
    text_fields=["question","text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [5]:
def search(query):
  boost = {'question':3.0, 'section':0.5}

  results = index.search(
      query = query,
      filter_dict ={'course': 'data-engineering-zoomcamp'},
      boost_dict = boost,
      num_results =5
  )
  return results

In [6]:
def rag(query):
  search_results = search(query)
  prompt = build_prompt(query, search_results)
  answer = llm(prompt)
  return answer

In [ ]:
from google.colab import userdata
userdata.get('myKey')

In [8]:
import os
from google.colab import userdata
from huggingface_hub import login

# Retrieve the user data
my_key = userdata.get('myKey')

# Check if the key was found and set it as an environment variable if it was.
if my_key:
  os.environ['myKey'] = my_key
  login(token=os.environ['myKey'])
else:
  print("Key 'myKey' not found in user data.")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /run/cache/token
Login successful


In [9]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [16]:

from transformers import pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
QUESTION: {question}

CONTEXT:
{context}

ANSWER:
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"{doc['question']}\n{doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = generator(prompt, max_length=500, temperature=0.7, top_p=0.95, num_return_sequences=1)
    response_final = response[0]['generated_text']
    return response_final[len(prompt):].strip()


In [18]:
rag("I just discovered the course. Can I still join it?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


''

<function __main__.rag(query)>